## 命令行推理

### For Windows

In [1]:
!chcp 65001

Active code page: 65001


### For Linux

In [2]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

## API

In [9]:
!python -m tools.api \
    --listen 0.0.0.0:8000 \
    --llama-checkpoint-path "checkpoints/fish-speech-1.2" \
    --decoder-checkpoint-path "checkpoints/fish-speech-1.2/firefly-gan-vq-fsq-4x1024-42hz-generator.pth" \
    --decoder-config-name firefly_gan_vq


^C


## For Test

### 0. 下载模型

In [3]:
!set HF_ENDPOINT=https://hf-mirror.com
# !export HF_ENDPOINT=https://hf-mirror.com
!huggingface-cli download fishaudio/fish-speech-1.2 --local-dir checkpoints/fish-speech-1.2/

D:\PythonProject\fish-speech\checkpoints\fish-speech-1.2



Fetching 8 files: 100%|██████████| 8/8 [00:00<00:00, 593.00it/s]


### 1. 从语音生成 prompt:
>  如果你打算让模型随机选择音色, 你可以跳过这一步.

你应该能得到一个 `fake.npy` 文件.

In [4]:
## 在此输入你的语音路径:
src_audio = r"D:\PythonProject\原神语音中文\胡桃\vo_hutao_draw_appear.wav"

!python tools/vqgan/inference.py \
    -i {src_audio} \
    --checkpoint-path "checkpoints/fish-speech-1.2/firefly-gan-vq-fsq-4x1024-42hz-generator.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)

2024-07-08 21:38:44.996 | INFO     | __main__:load_model:44 - Loaded model: <All keys matched successfully>
2024-07-08 21:38:44.997 | INFO     | __main__:main:73 - Processing in-place reconstruction of D:\PythonProject\原神语音中文\胡桃\vo_hutao_draw_appear.wav
2024-07-08 21:38:45.005 | INFO     | __main__:main:84 - Loaded audio with 19.05 seconds
2024-07-08 21:38:45.252 | INFO     | __main__:main:92 - Generated indices of shape torch.Size([4, 820])
2024-07-08 21:38:45.769 | INFO     | __main__:main:109 - Generated audio of shape torch.Size([1, 1, 839680]), equivalent to 19.04 seconds from 820 features, features/second: 43.07
2024-07-08 21:38:45.779 | INFO     | __main__:main:116 - Saved audio to fake.wav


### 2. 从语音生成 prompt:
> 该命令会在工作目录下创建 codes_N 文件, 其中 N 是从 0 开始的整数.

> 您可以使用 --compile 来融合 cuda 内核以实现更快的推理

In [7]:
!python tools/llama/generate.py \
    --text "人间灯火倒映湖中，她的渴望让静水泛起涟漪。若代价只是孤独，那就让这份愿望肆意流淌。流入她所注视的世间，也流入她如湖水般澄澈的目光。" \
    --prompt-text "唷，找本堂主有何贵干呀？嗯？你不知道吗，往生堂第七十七代堂主就是胡桃我啦！嘶，不过瞧你的模样，容光焕发，身体健康，嗯…想必是为了工作以外的事来找我，对吧？" \
    --prompt-tokens "fake.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.2" \
    --num-samples 2
    # --compile

2024-07-08 21:40:44.488 | INFO     | __main__:main:640 - Loading model ...
2024-07-08 21:40:52.870 | INFO     | __main__:load_model:347 - Restored model from checkpoint
2024-07-08 21:40:52.871 | INFO     | __main__:load_model:351 - Using DualARTransformer
2024-07-08 21:40:52.872 | INFO     | __main__:main:649 - Time to load model: 8.38 seconds
2024-07-08 21:40:52.909 | INFO     | __main__:generate_long:432 - Encoded text: 人间灯火倒映湖中,她的渴望让静水泛起涟漪.
2024-07-08 21:40:52.943 | INFO     | __main__:generate_long:432 - Encoded text: 若代价只是孤独,那就让这份愿望肆意流淌.
2024-07-08 21:40:52.944 | INFO     | __main__:generate_long:432 - Encoded text: 流入她所注视的世间,也流入她如湖水般澄澈的目光.
2024-07-08 21:40:52.944 | INFO     | __main__:generate_long:450 - Generating sentence 1/3 of sample 1/2
D:\PythonProject\fish-speech\fish_speech\models\text2semantic\llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\nati

### 3. 从语义 token 生成人声:

In [8]:
!python tools/vqgan/inference.py \
    -i "codes_0.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.2/firefly-gan-vq-fsq-4x1024-42hz-generator.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)

2024-07-08 21:42:28.650 | INFO     | __main__:load_model:44 - Loaded model: <All keys matched successfully>
2024-07-08 21:42:28.650 | INFO     | __main__:main:97 - Processing precomputed indices from codes_0.npy
2024-07-08 21:42:29.268 | INFO     | __main__:main:109 - Generated audio of shape torch.Size([1, 1, 666624]), equivalent to 15.12 seconds from 651 features, features/second: 43.07
2024-07-08 21:42:29.284 | INFO     | __main__:main:116 - Saved audio to fake.wav
